# SpaCy!

- huge library
- every thing, e.g., tagger, ner, pos, uses CNN
- #1 library for NLP (e.g., NLTK, gensim)
- HuggingFace (mostly for deep learning)

    `pip install spacy or pip install -U 'spacy[cuda-autodetect]'`

    `python -m spacy download en_core_web_sm`   #trained using cnn

    `python -m spacy download en_core_web_md`   #has word embedding (gloVe); trained using cnn

    `python -m spacy download en_core_web_trf`  #everything is trained using transformer

In [1]:
import spacy
spacy.__version__

2023-02-02 09:16:02.599371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'3.3.2'

### 1. Basics

In [2]:
#create a spacy object that can parse a lot of stuffs
#based on some learned model

nlp = spacy.load('en_core_web_sm')

In [10]:
text = 'Thailand really like to eat naan and masala.  He also likes to eat sushi.'

In [11]:
doc = nlp(text)

In [12]:
type(doc)

spacy.tokens.doc.Doc

In [14]:
#there are so many things in this doc
for tokens in doc[:10]:
    print(tokens)  #this spacy.tokens.doc.Doc already tokenize it!!!
    break

Thailand


In [15]:
tokens

Thailand

In [8]:
for sent in doc.sents:
    print(sent)  #it also has sentence 

Chaky really like to eat naan and masala.
 He also likes to eat sushi.


In [16]:
tokens

Thailand

In [17]:
tokens.ent_type #entity type ids

384

In [18]:
tokens.ent_type_ #geo political entity

'GPE'

In [19]:
spacy.explain('GPE')

'Countries, cities, states'

In [20]:
tokens.ent_iob_  #beginning of an entity

'B'

In [21]:
tokens.pos_  #proper noun

'PROPN'

In [22]:
tokens.dep_

'nsubj'

In [23]:
tokens.head

like

In [25]:
sentence1 = list(doc.sents)[0]

In [26]:
sentence1

Thailand really like to eat naan and masala.

In [28]:
from spacy import displacy  #displaying stuffs
displacy.render(sentence1, style="dep")

In [29]:
displacy.render(sentence1, style="ent")